## 数据清洗部分

### 目的：
- 尽可能将粉丝打榜、打卡抽奖、转载营销性质的微博和账号从数据集中排除
- 保留那些使用微博来生活日常、心情状态的用户
- 保留之后可能用的上的用户与微博信息字段，具体如下：
  - 用户：用户 ID、昵称、性别、总微博数、IP 所在地
  - 微博：微博 ID、用户 ID、微博文本、发布时间、点赞数、评论数、转发数

### 1. 微博有效性判断
- 这部分的清洗对象是**整条微博**
- 对于文本长度不超过1的微博，直接删除
- 对于文本中含有广告、转载、营销相关关键词的微博，直接删除

In [6]:
import re

def is_valid(text: str) -> bool:
    """
    判断该微博是否为有效微博，依据为广告、转载、营销相关的关键词
    如果微博中出现了关键词，则认为是无效微博，直接删除

    Args:
        text (str): 一条微博的文本内容

    Returns:
        bool: 微博是否是否有效
    """

    if len(text.strip()) <= 1:
        return False
    
    # 广告、转载相关关键词
    ad_keywords = [
        '点开红包', '现金红包', '好礼', '网页链接', 
        '我在参与', '连续签到', '粉打卡', '年度歌曲', 
        '免费围观', '关注超话', "蚂蚁庄园：", "森林驿站", 
        "头条文章", "注册微博", "注册微博", "闲鱼发布",
        "闲鱼号", "头像挂件"
    ]

    # 微博中含有任意一个关键词，则认为无效
    if any(kw in text for kw in ad_keywords):
        # print(f"微博“{text}”为抽奖、广告、转载等微博，删除")
        return False
    
    return True

### 2. 微博中低价值内容清洗
- 这部分的清洗对象是**一条微博中的部分内容**，具体做法是将相应的内容替换为空字符
- “低价值内容”指微博文本中的 URL、“分享自”、“分享图片/视频”和多余空格
- 相较于第一次项目：
  - 保留了“@用户”，猜测或许**能在一定程度上反映该用户的社交水平**
  - 保留了话题标签“#xxx#”，猜测或许能为**情绪事件的总结归纳**带来帮助

In [7]:

def clean_text(text):
    # 去除URL
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
    
    # 去除分享来源信息（直接将“分享自”及其后面的所有信息去除）
    text = re.sub(r'(?:[（(])?分享自(?!己).*$', '', text)

    # 去除“分享图片/视频”文本
    text = re.sub(r'分享(图片|视频)', '', text)
    
    # 去除多余空格
    text = re.sub(r'[\s]+', ' ', text)
    
    # 去除纯表情符号的内容
    if len(re.sub(r'[\s]', '', text)) <= 1:
        return ''
    
    return text.strip()

读取文件。这里事先从数据库的 `user` 表中提取前100个用户及其微博，作为清洗的测试样例

In [8]:
import json

file_name = "test100.json"

with open(file_name, 'r', encoding='utf-8') as f:
    user_dict:dict = json.load(f)

按上述策略执行清洗任务

In [9]:
users_to_remove = []

for user_id, user_info in user_dict.items():
    weibos = user_info["weibo"]
    for weibo in weibos:
        weibo["text"] = clean_text(weibo["text"])
    old = len(weibos)

    # 删除文本长度过短和抽奖打卡微博
    weibos = [weibo for weibo in weibos if is_valid(weibo["text"])]
    new = len(weibos)
    user_info["weibo"] = weibos
    rmv = old - new
    if new < 10:
        print(f"删除了用户{user_id}的{rmv}条无效微博，剩余微博数（{new}）过少，需要将该用户清洗出数据集")
        users_to_remove.append(user_id)
            
for user_id in users_to_remove:
    del user_dict[user_id]

print(f"删除了{len(users_to_remove)}个用户")

删除了用户1001148342的12条无效微博，剩余微博数（0）过少，需要将该用户清洗出数据集
删除了用户1011491753的21条无效微博，剩余微博数（5）过少，需要将该用户清洗出数据集
删除了用户1058548740的50条无效微博，剩余微博数（0）过少，需要将该用户清洗出数据集
删除了用户1144313674的48条无效微博，剩余微博数（2）过少，需要将该用户清洗出数据集
删除了4个用户


In [10]:
save_file = "test100cleaned.json"

with open(save_file, 'w', encoding='utf-8') as f:
    json.dump(user_dict, f, ensure_ascii=False, indent=4)

### 3. 使用大模型进行清洗
- 上面的清洗策略仅依据一些关键词和正则表达式识别低价值微博，但无法彻底清除掉那些粉丝打榜、转载营销的用户
- 接下来调用 DeepSeek 的 API，让大模型完成这部分用户的清洗工作


#### 微博文本采样
- DeepSeek 的 API 对长文本的接受度较高。为了提高清洗效率，这里在一次 API 的调用中，将多个用户的微博文本一同输入给大模型
- 为了节约时间和费用，下面先对各用户的微博进行采样
- 采样操作中对微博文本进行的操作仅作识别用户性质所用，并不会将这部分处理的文本用于后续的情感分析中
- 基本采样策略：
  - 微博内容预处理，去除话题标签、@用户、“微博视频”、日期部分
  - 舍弃长度小于10的微博；对于长度超过最大长度（此处设为50）的微博，保留前50字并在后加上 `...`
  - 将处理后的微博按长度排序，均匀抽取5条不同长度的微博，作为该用户的样本微博，供大模型分析所用

In [11]:
import random
from typing import List, Dict
import re

def sample_user_weibos(weibos: List[Dict], 
                      max_samples: int = 10,
                      min_samples: int = 3,
                      max_text_length: int = 50) -> str:
    """
    对用户的微博内容进行采样，用于LLM分析用户类型
    
    Args:
        weibos: 用户的微博列表，每个微博是一个字典，包含text等字段
        max_samples: 最大采样数量
        min_samples: 最小采样数量
        max_text_length: 每条微博的最大长度
    
    Returns:
        str: 采样后的微博文本，用换行符连接
    """
    def preprocess_text(text: str) -> str:
        # 去除话题标签 (#xxx#)
        text = re.sub(r'#.*?#', '', text)
        # 去除@用户
        text = re.sub(r'@\w+', '', text)
        # 去除“微博视频”部分
        text = re.sub(r'\s.+微博视频', '', text)
        # 去除日期部分
        text = re.sub(r'(\d{2,4}年\d{1,2}(月\d{1,2}(日)?)?|\d{1,2}月\d{1,2}日)', '', text)

        return text

    
    filtered_weibos = []
    for weibo in weibos:
        text = preprocess_text(weibo["text"])

        # 过滤掉太短的内容
        if len(text) < 10:
            continue

        # 截断过长的文本
        if len(text) > max_text_length:
            text = text[:max_text_length] + '...'

        filtered_weibos.append(text)
    
    # 如果过滤后的微博数量太少，可能说明这是个低价值用户
    if len(filtered_weibos) < min_samples:
        return '\n'.join(filtered_weibos)
    
    # 按微博的字数进行排序
    filtered_weibos.sort(key=lambda x: len(x))

    samples = []
    
    # 微博均匀采样
    if filtered_weibos:
        step = len(filtered_weibos) // (min(max_samples, len(filtered_weibos)) - 1)
        if step > 0:
            samples.extend(filtered_weibos[::step][:max_samples])
        else:
            samples.extend(random.sample(filtered_weibos, 
                                      min(max_samples-1, len(filtered_weibos))))
    
    return '\n'.join(samples)

将用户 ID 与采样后的微博文本组成一个元组，保存在一个字典中

In [12]:
sampled_weibo_dict = {}

for user_id, user_info in user_dict.items():
    sampled_weibo_dict[user_id] = sample_user_weibos(user_info["weibo"])

#### 设计 Prompt
- 这是最关键的部分，大模型的数据清洗效果很大程度上取决于 Prompt
- 大致思路是将 `sampled_weibo_dict` 输入给大模型，让大模型返回需要被清洗的用户 ID 组成的列表
- 一些实践：
  1. 为了评估大模型的清洗效果，我首先自己浏览了样本用户的微博内容，手动标注了我认为需要被清洗的用户的 ID，得到 `my_selected_list` 列表
  2. 对于 Prompt 的设计，一开始我采用了简短的描述，要求大模型筛选出没有表达出个人生活情感的用户。效果：容易遗漏，并且输出不稳定（时多时少）
  3. 接着，我将 `sampled_weibo_dict` 和 `my_selected_list` 输入给 DeepSeek-R1 模型，要求它分析我手动标注的这些用户微博内容，总结它们的共性特点，改善 API 的 Prompt，最终使用的 Prompt 如下：
   > 你是一名专业的社交媒体信息分析师，请根据用户发布的微博内容判断是否为生活分享账号，需满足以下至少2项特征：
   > 1. 包含具体生活场景/人物互动/个人经历
   > 2. 使用第一人称主观表达（如"我"的感受/经历）
   > 3. 内容呈现非结构化自然叙述（非列表/教程/资讯格式）
   > 4. 涉及日常活动（饮食/出行/家庭/宠物等）
   > 
   > 需排除以下特征账号：
   > - 垂直领域专业内容（医疗/法律/金融等）
   > - 商品交易/广告推广信息
   > - 抽象理论/鸡汤语录/政策转载
   > 
   > 接下来我将给出一个JSON文件，其中键是用户ID，值是经过采样后的微博内容。
   > 
   > 请直接为我返回一个列表，其中的元素是需要排除的用户的ID，除此之外不需要给出其他信息。
   > 
   > 返回示例：[1234567890, 2345678901, ...]

- 采用改善后的 Prompt，大模型的输出质量有所提高，不过仍会存在输出不稳定的情况
- 于是，提出一个想法：让大模型多次分析，综合结果来决定清洗方式。具体做法：
  - 让大模型连续分析3次，得到3个清洗用户的列表
  - 对列表中的 ID 进行计数：对于出现3次或2次的用户，可以直接从数据集中排除；对于只出现了1次的用户，则人工检查和决定清洗或保留

In [14]:
from openai import OpenAI

with open("../weibo_config.json", 'r', encoding='utf-8') as f:
    config = json.load(f)



prompt = """
你是一名专业的社交媒体信息分析师，请根据用户发布的微博内容判断是否为生活分享账号，需满足以下至少2项特征：
1. 包含具体生活场景/人物互动/个人经历
2. 使用第一人称主观表达（如"我"的感受/经历）
3. 内容呈现非结构化自然叙述（非列表/教程/资讯格式）
4. 涉及日常活动（饮食/出行/家庭/宠物等）

需排除以下特征账号：
• 垂直领域专业内容（医疗/法律/金融等）
• 商品交易/广告推广信息
• 抽象理论/鸡汤语录/政策转载
接下来我将给出一个JSON文件，其中键是用户ID，值是经过采样后的微博内容。
请直接为我返回一个列表，其中的元素是需要排除的用户的ID，除此之外不需要给出其他信息。
返回示例：[1234567890, 2345678901, ...]
"""

def validity_analysis(api_config: dict, prompt: str, sampled_weibo):
    base_url = api_config["base_url"]
    model = api_config["model"]
    api_key = api_config["api_key"]
    if isinstance(api_key, list):
        api_key = api_key[0]
    if not isinstance(sampled_weibo, str):
        if isinstance(sampled_weibo, dict):
            sampled_weibo = json.dumps(sampled_weibo, ensure_ascii=False)
        else:
            sampled_weibo = str(sampled_weibo)
        
    client = OpenAI(api_key=api_key, base_url=base_url)


    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": sampled_weibo},
        ],
        temperature=1.0, 
        stream=False
    )

    return response.choices[0].message.content


# 读取配置文件
with open(r"../weibo_config.json", 'r', encoding='utf-8') as f:
    config = json.load(f)

llm_config = config["LLM_API"]["DeepSeek-v3"]


In [114]:
import time
from collections import Counter

outputs = []
# 连续3次分析
for i in range(3):
    start = time.time()
    output = validity_analysis(llm_config, prompt, sampled_weibo_dict)
    end = time.time()
    print(f"第 {i + 1} 次分析，用时 {end - start:.2f} 秒")
    outputs.append(output)

selected_user_list = []
for output in outputs:
    user_list = output[1:-1].split(', ')
    user_list = [user.strip('"').strip("'") for user in user_list]
    selected_user_list.extend(user_list)

user_counter = Counter(selected_user_list)

第 1 次分析，用时 12.40 秒
第 2 次分析，用时 13.86 秒
第 3 次分析，用时 20.74 秒


#### 大模型分析后的处理
- 上面向大模型输入的用户数约为100
- 经过多次重复测试，大模型分析的用时基本在10~20秒，取平均值15秒，则大模型分析一个用户是否应该被清洗的平均用时约为
$$\frac{15\times 3}{100}=0.45\text{s}\approx 0.5\text{s}$$
- 因此，如果分析完收集的约15000个用户的有效性，用时约为
$$15000\times 0.5 = 75000\text{s}\approx21\text{h}$$

In [115]:
cleaned_user_list = [user for user, freq in user_counter.items() if freq >= 2]
manual_examine_user_list = [user for user, freq in user_counter.items() if freq == 1]

cleaned_count = len(cleaned_user_list)
manual_count = len(manual_examine_user_list)
total_count = len(user_dict)


print(f"直接删除用户数：{cleaned_count}；人工筛选用户数：{manual_count}")
print(f"直接删除比例：{cleaned_count/total_count:.2f}；人工筛选比例：{manual_count/total_count:.2f}")

直接删除用户数：33；人工筛选用户数：7
直接删除比例：0.34；人工筛选比例：0.07


后续批量处理时，可以将需要人工检查的那部分用户 ID 专门存于一个文本文档中。

同时，后续有可能需要再次检查被大模型清洗掉的那部分用户（数据量不足时），因此，可以将出现次数等于2的用户单独记录在一个文本文件中

In [117]:
manual_examine_file = "manual_examine.txt"
cleaned_user_file = "cleaned_users_2.txt"  # 2 代表出现次数等于2

with open(manual_examine_file, 'a', encoding='utf-8') as f:
    f.writelines('\n'.join(manual_examine_user_list))

with open(cleaned_user_file, 'a', encoding='utf-8') as f:
    f.writelines('\n'.join(user for user, freq in user_counter.items() if freq == 2))

- 人工筛选的比例基本在5%~10%左右，取平均值7.5%。故对于15000个总用户来说，需要手动筛选的用户数约为
$$15000\times 7.5\% = 1125$$
- 下面手动检查这7人的微博，方法是从他们未经采样处理的微博中随机收集10条微博，检查这些微博是否表露出情感倾向

In [116]:
for user_id in manual_examine_user_list:
    weibo_list = user_dict[user_id]["weibo"]
    sampled_weibo_list = random.sample(weibo_list, 10)
    print(f"{user_id}:")
    for i, weibo in enumerate(sampled_weibo_list):
        text = weibo["text"]
        print(f'({i + 1}) {text[:min(50, len(text))]}', end='')
        print('...' if len(text) > 50 else '')
    print('-'*50)

1136167331:
(1) #生活手记#今日早餐：百香果茶、煎蛋香肠、土豆丝煎饼
(2) #生活手记#今日早餐：百合山药水、煎蛋肉沫炒荞 新入手一大箱子荞麦方便面，嗯，想着来两块儿面饼吧，毕...
(3) #生活手记#今日早餐：西红柿菠菜豆腐汤
(4) #生活手记#今日早餐：红枣山药水、韭菜盒子
(5) #生活手记#今天仍然一餐，昨天的帕尼尼一个，韭菜肉馅儿馄饨一碗。韭菜再不吃就要坏掉了，馄饨皮也是 。...
(6) #生活手记#今日早餐：发面单饼、酸辣粉 前两天打豆浆，花生黄豆豆浆真的很香。破壁机打的，剩下的豆渣就...
(7) #生活手记#今日早餐：牛奶、鸡蛋酱吐司、烤肠
(8) #生活手记#今日早餐：杂粮粥、芹菜炒肉片 一天一锅粥，也不是不行哈 ~但我刚才去泡了牛奶燕麦，所以说...
(9) #生活手记#今日早午餐：橙皮百香果汁茶、鲜肉小馄饨、韭菜盒子
(10) #生活手记#今日早餐：野菜包子、胡辣汤
--------------------------------------------------
1141951792:
(1) #生活手记#换了拉花缸，换了杯型，拉花又不会了
(2) #生活手记#许久未用的杯子 还是很养眼哈
(3) #生活手记#追日🌄
(4) #生活手记#早安☕️
(5) #生活手记#抹茶🍵拿铁 卖相不行 味道不错
(6) #生活手记#元宵节快乐
(7) #生活手记#早安
(8) #生活手记#coffee everyday ☕️
(9) #生活手记#浪里个浪 上海·古猗庄园
(10) #生活手记#阳台种菜愿望初步达成
--------------------------------------------------
1144681142:
(1) #猫猫的奇幻漂流奥斯卡最佳动画长片#恭喜猫猫
(2) #刘宇宁海飞丝活力代言人# #刘宇宁开始推理吧# 摩登兄弟 知所从来，思所将往方明所去，我还没编好说...
(3) #刘宇宁燕子京2024微博热议男角色#所以说嘛刘宇宁的剧，必看 摩登兄弟
(4) 春困开始了
(5) 熟悉的配方0.5+1+2
(6) #刘宇宁海飞丝活力代言人# #刘宇宁开始推理吧# 摩登兄弟 好好好，就欢喜你这般自有规划的独立意志，...
(7) #刘宇宁维护工作室#这才是成熟男人该做的事
(8) #刘宇宁开始推理吧#

从这7人中，筛选出以下3人保留：['1147177281', '1171498500', '1173559413']

In [120]:
valid_user_list = ['1147177281', '1171498500', '1173559413']

cleaned_user_list.extend([user for user in manual_examine_user_list 
                          if user not in valid_user_list])

for user_id in cleaned_user_list:
    user_dict.pop(user_id)

最后，将清洗后的数据集存入数据库的 `user_cleaned` 表和 `weibo_cleaned` 表中，进行后续的情感分析。

In [125]:
import pymysql

# 读取配置文件
with open('weibo_config.json', 'r') as f:
    config = json.load(f)

In [127]:
user_list = []
weibo_list = []

for user_id, user_info in user_dict.items():
    user_data = {
        "id": user_id, 
        "screen_name": user_info["screen_name"], 
        "sex": user_info["sex"], 
        "statuses_count": user_info["statuses_count"], 
        "IP": user_info["IP"]
    }
    user_list.append(user_data)

    for weibo in user_info["weibo"]:
        created_at = weibo["created_at"].replace('T', ' ')

        weibo_data = {
            "weibo_id": weibo["weibo_id"], 
            "user_id": user_id, 
            "text": weibo["text"], 
            "created_at": created_at, 
            "attitudes_count": weibo["attitudes_count"], 
            "comments_count": weibo["comments_count"], 
            "reposts_count": weibo["reposts_count"]
        }
        weibo_list.append(weibo_data)

with pymysql.connect(**config['database']) as conn:
    try:
        with conn.cursor() as cursor:
            user_sql = """
                INSERT INTO user_cleaned
                (uid, screen_name, sex, statuses_count, IP)
                VALUES
                (%(id)s, %(screen_name)s, %(sex)s, %(statuses_count)s, %(IP)s)
            """
            cursor.executemany(user_sql, user_list)

            weibo_sql = """
                INSERT INTO weibo_cleaned
                (wid, user_id, text, created_at, 
                attitudes_count, comments_count, reposts_count)
                VALUES
                (%(weibo_id)s, %(user_id)s, %(text)s, %(created_at)s, 
                %(attitudes_count)s, %(comments_count)s, %(reposts_count)s)
            """
            cursor.executemany(weibo_sql, weibo_list)

        conn.commit()
        print(f"数据插入成功，共插入 {len(user_list)} 个用户，{len(weibo_list)} 条微博")
    
    except Exception as e:
        conn.rollback()
        print(f"数据插入失败，已回滚：{e}")
        raise

数据插入成功，共插入 59 个用户，2468 条微博


- 经过清洗阶段，从100个用户得到59个有效用户，有效率约为60%
- 因此，对于15000个总用户，最终得到的有效用户数约为
$$15000 \times 60\% = 9000$$

将出现2次的用户及其微博组成字典，存于JSON文件，便于检查

In [8]:
import json

with open("cleaned_users_2.txt", 'r', encoding='utf-8') as f:
    cleaned_users_2 = f.readlines()
    cleaned_users_2 = [user.strip() for user in cleaned_users_2]

cleaned_users_2_weibo = {}
for user_id in cleaned_users_2:
    user_info = user_dict[user_id]
    weibo_text_list = [weibo["text"] for weibo in user_info["weibo"]]
    cleaned_users_2_weibo.update({user_id: weibo_text_list})



with open("cleaned_user_2_weibo.json", 'w', encoding='utf-8') as f:
    json.dump(cleaned_users_2_weibo, f, ensure_ascii=False, indent=4)

In [2]:
import json

with open("test100cleaned.json", 'r', encoding='utf-8') as f:
    user_dict:dict = json.load(f)

new_dict = {}

for user_id, user_info in user_dict.items():
    user = user_info
    user["weibo"] = [weibo["text"] for weibo in user_info["weibo"]]
    new_dict[user_id] = user

with open("test100_Cleaned.json", 'w', encoding='utf-8') as f:
    json.dump(new_dict, f, ensure_ascii=False, indent=4)

In [5]:
with open("examine_done.txt", 'r', encoding='utf-8') as f:
    users = f.readlines()

users = list(set(users))

with open("examine_done.txt", 'w', encoding='utf-8') as f:
    f.writelines(users)

In [6]:
def get_users_to_examine(examine_file: str, done_file: str):
    with open(examine_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        users_to_examine = [line.strip() for line in lines]

    with open(done_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        users_done = [line.strip() for line in lines]

    users_to_examine = [user for user in users_to_examine if user not in users_done]
    print(f"已获取检查用户，共 {len(users_to_examine)} 个，已检查 {len(users_done)} 个")
    return users_to_examine


examine_file = "users_to_examine.txt"
done_file = "examine_done.txt"
user_list = get_users_to_examine(examine_file, done_file)

user_list

已获取检查用户，共 691 个，已检查 60 个


['1057539901',
 '1711730301',
 '1711774640',
 '1711906544',
 '1720055177',
 '1734465322',
 '1739636063',
 '1761440614',
 '1762901112',
 '1768206720',
 '1771443730',
 '1772112613',
 '1772483351',
 '1772928614',
 '1780368803',
 '1799090591',
 '1803889674',
 '1805741880',
 '1806775502',
 '1808074104',
 '1823093857',
 '1823492827',
 '1828290917',
 '1832068381',
 '1832181315',
 '1834023444',
 '1849065705',
 '1851397621',
 '1853004904',
 '1857768742',
 '1858284742',
 '1863792341',
 '1867573791',
 '1867682701',
 '1868043311',
 '1876259453',
 '1877347654',
 '1882547907',
 '1885434317',
 '1887927493',
 '1893748821',
 '1897856565',
 '1908848893',
 '1912015121',
 '1912962573',
 '1915924712',
 '1917048154',
 '1918774482',
 '1922360840',
 '1934801684',
 '1942536853',
 '1948039074',
 '1949142934',
 '1951113427',
 '1952116440',
 '1955020080',
 '1956894162',
 '1958570917',
 '1962018051',
 '1965552242',
 '1971797907',
 '1971881973',
 '1989357921',
 '1991396037',
 '1996673125',
 '2001911524',
 '20021631